In [3]:
# modified from merkle tree basic implementation from https://github.com/andipro123/merkle-tree-Python/blob/master/merkleTree.py

import hashlib

#function to read in text files and store in a flat list
def collect_files():
    file_list = []
    done = False
    while done == False:
        file_name = input("Please enter file name and path or type 'done': ")
        if file_name == 'done':
            done = True
        else:
            with open(file_name, 'r') as file:
                file_object = file.read()
                file_list.append(file_object)
    print("Done with list")
    return(file_list)

#function to alter text in one of the input files
def change_text():
    file_name = input("Please enter file name and path to change: ")
    with open (file_name, 'w') as file:
        new_text = input("Please enter the new text: ")
        file.write(new_text)
    return()

#make two lists from reading in files, file_list2 will read in one altered text
file_list1 = collect_files()
change_text()
file_list2 = collect_files()

#test flat input list 
quicktestlist = ['test1', 'test2', 'test3', 'test4', 'test5', 'test6']
#for testing verifyUtil function
quickverifytestlist = ['test1', 'test2', 'test3', 'test4', 'test5', 'test7']

# Node class that represents a node of a binary tree
class Node:
    #function to initialise the node with data, left child and right child, representing a binary
    #tree
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
    
    #utility function to check if a given node has 2 children
    def isFull(self):
        return self.left and self.right
    
    #utility function to print the contents of the node
    def __str__(self):
        return self.data
    
    #utility function to check if a given node has 0 children (is a leaf node)
    def isLeaf(self):
        return ((self.left == None) and (self.right == None))

# Merkle tree class for actual implementation of the tree
class merkleTree:

    # function to initialise the tree with "None" as root and merkleRoot set to blank
    def __init__(self, arr):
        self.root = None
        self._merkleRoot = ''
        self.makeTreeFromArray(arr)
        self.calculateMerkleRoot()
        
    # private utility function to return a hash of an input string, using the sha256 algorithm    
    def __returnHash(self, x):
            return (hashlib.sha256(x.encode()).hexdigest())
        
    # function to generate a tree from a list. The list is expected to contain
    # the list of transactions(as strings) we wish to seal in a block    
    def makeTreeFromArray(self, arr):
        arr = arr.copy()
        # all the elements of the list must be leaf nodes, hence total nodes in the tree
        # is [2*(noOfLeafNodes) - 1]
        def __noOfNodesReqd(arr):
            x = len(arr)
            return (2*x - 1)
        
        # Private function to build tree with a list generated by a sequence corresponding to the
        # number of nodes required for the tree.
        # For e.g. if we have 4 transactions, i.e. 4 leafNodes, we will have a total of 7 nodes
        # in the tree. Hence this function takes in a list that looks like [1,2,3,4,5,6,7] and 
        # creates a binary tree by recursively inserting elements in inorder fashion.
        def __buildTree(arr, root, i, n): 
      
            # Base case for recursion  
            if i < n: 
                temp = Node(str(arr[i]))  
                root = temp  

                # insert left child  
                root.left = __buildTree(arr, root.left,2 * i + 1, n)  

                # insert right child  
                root.right = __buildTree(arr, root.right,2 * i + 2, n) 
                
            return root
        
        # This function adds our transactions (leaf Node data) to the Leaf Nodes in the tree
        # hence now the tree looks like :
                #                 ""
                #                 /\
                #         ""              ""
                #         /\              /\
                #    (txn1) (txn2)  (txn3) (txn4)      
        def __addLeafData(arr, node):
            
            if not node:
                return
            
            __addLeafData(arr, node.left)
            if node.isLeaf():
                node.data = self.__returnHash(arr.pop())
            else:
                node.data = ''
            __addLeafData(arr, node.right)
        
        #Driver function calls of the main function makeTreeFromArray
        nodesReqd = __noOfNodesReqd(arr)
        nodeArr = [num for num in range(1,nodesReqd+1)]
        self.root = __buildTree(nodeArr, None, 0, nodesReqd)
        __addLeafData(arr,self.root)

    # utility function to traverse the tree using inorder Traversal algorithm, must provide <the_name_of_the_tree_object>.root     
    def inorderTraversal(self, node):
        if not node:
            return
        
        self.inorderTraversal(node.left)
        print(node)
        self.inorderTraversal(node.right)
    
    # function to calculate merkle root of the tree. This is a recursive algorithm.
    # Essentially, the data of parent node P, is the hash of concatenation of data of its two 
    # child nodes, A and B. If H(x) is the hash function,
    # P = H( H(A) + H(B) )
    # If R is the root, then data of R is the merkle hash or merkle root.  
    def calculateMerkleRoot(self):
         
        def __merkleHash(node):
            if node.isLeaf():
                return node
            
            left = __merkleHash(node.left).data
            right = __merkleHash(node.right).data
            node.data = self.__returnHash(left+right)
            return node
        
        merkleRoot = __merkleHash(self.root)
        self._merkleRoot = merkleRoot.data
        
        return self._merkleRoot 

    #utility function to get the merkle Root of that tree
    def getMerkleRoot(self):
        return self._merkleRoot
    
    # utility function to verify if the transactions are intact with respect to this tree.
    # say we know transaction list A is intact and original. We create a merkle Tree with that.
    # Now we want to know whether the original list of transactions has been tampered with,
    # this function takes in a new list, creates a new merkle tree, calculate its merkle root,
    # and returns if merkle roots match. If they match, transaction list is intact and verified.
    # If transactions are tampered or changed, this function returns False.
    def verifyUtil(self, arr1):
        hash1 = self.getMerkleRoot()
        new_tree = merkleTree(arr1)
        hash2 = new_tree.getMerkleRoot()
        if hash1 == hash2 :
            print("Texts verified successfully")
            return True
        else:
            print("Texts have been changed")
            return False

testtree = merkleTree(file_list1)
testtree.inorderTraversal(testtree.root)
#compare file_list1 against itself 
test1 = testtree.verifyUtil(file_list1)
#compare file_list2 against file_list1
test2 = testtree.verifyUtil(file_list2)







Please enter file name and path or type 'done':  project4part3question1testinput1.txt
Please enter file name and path or type 'done':  project4part3question1testinput2.txt
Please enter file name and path or type 'done':  project4part3question1testinput3.txt
Please enter file name and path or type 'done':  project4part3question1testinput4.txt
Please enter file name and path or type 'done':  done


Done with list


Please enter file name and path to change:  project4part3question1testinput4.txt
Please enter the new text:  sharknado!
Please enter file name and path or type 'done':  project4part3question1testinput1.txt
Please enter file name and path or type 'done':  project4part3question1testinput2.txt
Please enter file name and path or type 'done':  project4part3question1testinput3.txt
Please enter file name and path or type 'done':  project4part3question1testinput4.txt
Please enter file name and path or type 'done':  done


Done with list
220ffe36f1594151c30f11693969a6fdfff38ac16002cf3e4e9a47ead3a66f98
b706043742a5f96dd9369f668e57928394dc8d874b3ba66bf716730d638a3360
fd61a03af4f77d870fc21e05e7e80678095c92d808cfb3b5c279ee04c74aca13
e5b565c1e9b014b553e64a654998e0b0c8e743a6d78289e1531c37ab37c28a12
60303ae22b998861bce3b28f33eec1be758a213c86c93c076dbe9f558c11c752
254829b2cd5a1d6e0b4796bf875468e57e544573f8b504115b836e477fcf9596
1b4f0e9851971998e732078544c96b36c3d01cedf7caa332359d6f1d83567014
Transactions verified successfully
Transactions have been changed
